
## **Set 2: Sensor Integration and Reactive Behaviors**

### 11. **Task 11**: Use multiple proximity sensors to detect and avoid obstacles in different directions.


In [4]:
import sim  # Import the CoppeliaSim remote API
import time  # Import the time library for delays

# Step 1: Connect to CoppeliaSim
print("Connecting to CoppeliaSim...")
client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)
if client_id == -1:
    print("Failed to connect to CoppeliaSim. Please ensure the simulator is running.")
    exit()
else:
    print(f"Connected to CoppeliaSim! Client ID: {client_id}")

# Step 2: Get motor and sensor handles
print("Retrieving motor and sensor handles...")
_, left_motor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_leftMotor', sim.simx_opmode_blocking)
_, right_motor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_rightMotor', sim.simx_opmode_blocking)

# Proximity sensors for all directions
_, front_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor5', sim.simx_opmode_blocking)
_, left_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor2', sim.simx_opmode_blocking)
_, right_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor8', sim.simx_opmode_blocking)
_, back_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor1', sim.simx_opmode_blocking)

if left_motor_handle == -1 or right_motor_handle == -1 or front_sensor_handle == -1:
    print("Error: Could not retrieve motor or sensor handles. Exiting...")
    sim.simxFinish(client_id)
    exit()
else:
    print("Motor and sensor handles retrieved successfully.")

# Step 3: Start streaming data from all sensors
print("Starting proximity sensor streams...")
sensors = {
    "front": front_sensor_handle,
    "left": left_sensor_handle,
    "right": right_sensor_handle,
    "back": back_sensor_handle,
}
for direction, handle in sensors.items():
    sim.simxReadProximitySensor(client_id, handle, sim.simx_opmode_streaming)
time.sleep(0.5)  # Allow sensor streams to initialize

# Step 4: Define motion parameters
forward_speed = 10  # Speed in meters per second
turn_speed = 0.5  # Turning speed for obstacle avoidance
reverse_speed = -0.5  # Speed for reversing
safety_distance = 0.5  # Distance threshold for obstacle detection (meters)

print(f"Motion parameters:\n - Forward speed: {forward_speed}\n - Turn speed: {turn_speed}\n - Safety distance: {safety_distance} meters")

# Step 5: Main control logic
try:
    while True:
        # Step 5.1: Read data from all sensors
        distances = {}
        for direction, handle in sensors.items():
            _, detection_state, detected_point, _, _ = sim.simxReadProximitySensor(client_id, handle, sim.simx_opmode_buffer)
            if detection_state:
                distances[direction] = (detected_point[0]**2 + detected_point[1]**2 + detected_point[2]**2)**0.5
                print(f"{direction.capitalize()} sensor detected an obstacle at distance: {distances[direction]:.2f} meters.")
            else:
                distances[direction] = None
                print(f"{direction.capitalize()} sensor detected no obstacle.")

        # Step 5.2: Obstacle avoidance logic
        if distances["front"] and distances["front"] < safety_distance:
            print("Obstacle detected in front. Checking alternative paths...")

            if distances["left"] and distances["left"] > safety_distance:
                print("Space available on the left. Turning left...")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, -turn_speed, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, turn_speed, sim.simx_opmode_blocking)
                time.sleep(1.0)
            elif distances["right"] and distances["right"] > safety_distance:
                print("Space available on the right. Turning right...")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, turn_speed, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, -turn_speed, sim.simx_opmode_blocking)
                time.sleep(1.0)
            elif distances["back"] and distances["back"] > safety_distance:
                print("No space on the sides. Reversing...")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, reverse_speed, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, reverse_speed, sim.simx_opmode_blocking)
                time.sleep(1.0)
            else:
                print("No space available. Stopping to avoid collision.")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, 0, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, 0, sim.simx_opmode_blocking)

            continue  # Reassess environment after avoidance action

        # Move forward if no obstacles are detected
        if not distances["front"] or distances["front"] > safety_distance:
            print("Path is clear. Moving forward...")
            sim.simxSetJointTargetVelocity(client_id, left_motor_handle, forward_speed, sim.simx_opmode_blocking)
            sim.simxSetJointTargetVelocity(client_id, right_motor_handle, forward_speed, sim.simx_opmode_blocking)

        time.sleep(0.1)  # Short delay for smoother control

except KeyboardInterrupt:
    print("Program interrupted by user. Stopping the robot...")

# Step 6: Stop the robot and disconnect
print("Stopping robot and closing connection to CoppeliaSim...")
sim.simxSetJointTargetVelocity(client_id, left_motor_handle, 0, sim.simx_opmode_blocking)
sim.simxSetJointTargetVelocity(client_id, right_motor_handle, 0, sim.simx_opmode_blocking)
sim.simxFinish(client_id)
print("Disconnected from CoppeliaSim.")


Connecting to CoppeliaSim...
Connected to CoppeliaSim! Client ID: 0
Retrieving motor and sensor handles...
Motor and sensor handles retrieved successfully.
Starting proximity sensor streams...
Motion parameters:
 - Forward speed: 10
 - Turn speed: 0.5
 - Safety distance: 0.5 meters
Front sensor detected no obstacle.
Left sensor detected an obstacle at distance: 0.72 meters.
Right sensor detected no obstacle.
Back sensor detected an obstacle at distance: 0.98 meters.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected an obstacle at distance: 0.69 meters.
Right sensor detected no obstacle.
Back sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected an obstacle at distance: 0.66 meters.
Right sensor detected no obstacle.
Back sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected an obstacle at distance: 0.65 meters.
Right sens

### 12. **Task 12**: Implement edge detection using proximity sensors to avoid falling off a surface.


In [16]:
import sim  # Import the CoppeliaSim remote API
import time  # Import the time library for delays
import random  # Import the random library to choose random turns

# Step 1: Connect to CoppeliaSim
print("Connecting to CoppeliaSim...")
client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)
if client_id == -1:
    print("Failed to connect to CoppeliaSim.")
    exit()
else:
    print(f"Connected to CoppeliaSim! Client ID: {client_id}")

# Step 2: Get motor and sensor handles
print("Retrieving motor and sensor handles...")
_, left_motor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_leftMotor', sim.simx_opmode_blocking)
_, right_motor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_rightMotor', sim.simx_opmode_blocking)

# Using the front, left, and right proximity sensors for edge detection.
_, front_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor5', sim.simx_opmode_blocking)
_, left_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor2', sim.simx_opmode_blocking)
_, right_sensor_handle = sim.simxGetObjectHandle(client_id, 'Pioneer_p3dx_ultrasonicSensor8', sim.simx_opmode_blocking)

if left_motor_handle == -1 or right_motor_handle == -1 or front_sensor_handle == -1 or left_sensor_handle == -1 or right_sensor_handle == -1:
    print("Error: Could not retrieve motor or sensor handles. Exiting...")
    sim.simxFinish(client_id)
    exit()
else:
    print("Motor and sensor handles retrieved successfully.")

# Step 3: Start streaming data from the sensors
print("Starting proximity sensor streams...")
sim.simxReadProximitySensor(client_id, front_sensor_handle, sim.simx_opmode_streaming)
sim.simxReadProximitySensor(client_id, left_sensor_handle, sim.simx_opmode_streaming)
sim.simxReadProximitySensor(client_id, right_sensor_handle, sim.simx_opmode_streaming)
time.sleep(0.5)  # Allow sensor streams to initialize

# Step 4: Define motion parameters
forward_speed = 1  # Speed for moving forward
turn_speed = 0.5  # Turning speed
reverse_speed = -1  # Speed for reversing
safety_distance = 1  # Distance threshold to detect obstacles/edges (meters)

print(f"Motion parameters:\n - Forward speed: {forward_speed}\n - Turn speed: {turn_speed}\n - Safety distance: {safety_distance} meters")

# Step 5: Main control logic
try:
    while True:
        # Step 5.1: Read data from all sensors
        distances = {}
        sensors = {
            "front": front_sensor_handle,
            "left": left_sensor_handle,
            "right": right_sensor_handle
        }
        for direction, handle in sensors.items():
            _, detection_state, detected_point, _, _ = sim.simxReadProximitySensor(client_id, handle, sim.simx_opmode_buffer)
            if detection_state:
                distances[direction] = (detected_point[0]**2 + detected_point[1]**2 + detected_point[2]**2)**0.5
                print(f"{direction.capitalize()} sensor detected an obstacle at distance: {distances[direction]:.2f} meters.")
            else:
                distances[direction] = None
                print(f"{direction.capitalize()} sensor detected no obstacle.")

        # Step 5.2: Obstacle avoidance logic
        if distances["front"] and distances["front"] < safety_distance:
            print("Obstacle detected in front. Reversing and then turning...")

            # Reverse for a short distance
            sim.simxSetJointTargetVelocity(client_id, left_motor_handle, reverse_speed, sim.simx_opmode_blocking)
            sim.simxSetJointTargetVelocity(client_id, right_motor_handle, reverse_speed, sim.simx_opmode_blocking)
            time.sleep(1.0)  # Adjust the time for reversing as needed to achieve 1 meter

            # Randomly choose to turn left or right
            turn_direction = random.choice(["left", "right"])
            if turn_direction == "left":
                print("Turning left 90 degrees...")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, -turn_speed, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, turn_speed, sim.simx_opmode_blocking)
            else:
                print("Turning right 90 degrees...")
                sim.simxSetJointTargetVelocity(client_id, left_motor_handle, turn_speed, sim.simx_opmode_blocking)
                sim.simxSetJointTargetVelocity(client_id, right_motor_handle, -turn_speed, sim.simx_opmode_blocking)

            time.sleep(1.0)  # Adjust time to complete a 90-degree turn
            continue  # Reassess environment after avoidance action

        # Move forward if no obstacles are detected
        if not distances["front"] or distances["front"] > safety_distance:
            print("Path is clear. Moving forward...")
            sim.simxSetJointTargetVelocity(client_id, left_motor_handle, forward_speed, sim.simx_opmode_blocking)
            sim.simxSetJointTargetVelocity(client_id, right_motor_handle, forward_speed, sim.simx_opmode_blocking)

        time.sleep(0.1)  # Short delay for smoother control

except KeyboardInterrupt:
    print("Program interrupted by user. Stopping the robot...")

# Step 6: Stop the robot and disconnect
print("Stopping robot and closing connection to CoppeliaSim...")
sim.simxSetJointTargetVelocity(client_id, left_motor_handle, 0, sim.simx_opmode_blocking)
sim.simxSetJointTargetVelocity(client_id, right_motor_handle, 0, sim.simx_opmode_blocking)
sim.simxFinish(client_id)
print("Disconnected from CoppeliaSim.")


Connecting to CoppeliaSim...
Connected to CoppeliaSim! Client ID: 0
Retrieving motor and sensor handles...
Motor and sensor handles retrieved successfully.
Starting proximity sensor streams...
Motion parameters:
 - Forward speed: 1
 - Turn speed: 0.5
 - Safety distance: 1 meters
Front sensor detected no obstacle.
Left sensor detected no obstacle.
Right sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected no obstacle.
Right sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected no obstacle.
Right sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected no obstacle.
Right sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.
Left sensor detected no obstacle.
Right sensor detected no obstacle.
Path is clear. Moving forward...
Front sensor detected no obstacle.


### 13. **Task 13**: Integrate a light sensor and follow a light source.


In [ ]:
# Note: I couldnt create the environment in this

### 14. **Task 14**: Follow a line on the ground using a color or line-following sensor.


### 15. **Task 15**: Implement object-following behavior (follow an object at a constant distance).


### 16. **Task 16**: Avoid dynamic obstacles (objects moving in the robot's path).


### 17. **Task 17**: Use infrared sensors to maintain a safe distance from obstacles.

it uses Test Set 02 Scene.ttt


### 18. **Task 18**: Create a simple mapping of the environment using a proximity sensor.


### 19. **Task 19**: Combine light-following and obstacle-avoidance behaviors.


### 20. **Task 20**: Implement simple goal-directed navigation using waypoints.

